# Init

In [37]:
import pandas as pd
import geopandas as gpd
import pandasql as ps
from datetime import datetime,date
from shapely.geometry import *
import geopandas as gpd
from shapely.ops import nearest_points

from codes.db import * 
from codes.lib import *
from codes.riverlog import *
from codes.flwdir import *
import codes.wflow as wflow

# pandas + pandasql 

In [12]:
import pandas as pd 
import pandasql as ps
# 展示方便，檔案請先手動下載至特定位置
data_df = pd.read_csv("data/22371-工業區污水處理廠放流水水質資訊.csv")
print("data_df:\n%s" %(data_df.head(2)))

find_df = data_df[data_df['廠名'].str.contains('新竹')][['日期','廠名','導電度( μscm)']]
print("find_df:\n%s" %(find_df.head(2)))


query_str = """SELECT * FROM data_df where 廠名 like '%新竹%'"""
query_df= ps.sqldf(query_str, locals())
print("query_df:\n%s" %(query_df.head(2)))

find_df

data_df:
         日期    廠名  導電度( μscm)  氫離子濃度指標  水溫(℃)
0  20190926    土城        5289     6.48  29.69
1  20190926  大甲幼獅        8439     6.88  27.17
find_df:
          日期  廠名  導電度( μscm)
26  20190926  新竹        5084
61  20190927  新竹        5271
query_df:
         日期  廠名  導電度( μscm)  氫離子濃度指標  水溫(℃)
0  20190926  新竹        5084     6.90  28.43
1  20190927  新竹        5271     6.92  28.67


,日期,廠名,導電度( μscm)
26,20190926,新竹,5084
61,20190927,新竹,5271
96,20190928,新竹,5464
131,20190929,新竹,5090
166,20190930,新竹,4936
...,...,...,...
20044,20210411,新竹,5390
20080,20210412,新竹,5142
20116,20210413,新竹,4766
20152,20210414,新竹,3436


# Data Transfer Kit

In [ ]:
import os,fnmatch
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        files.sort()
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result


if 0: # 多行轉成一行，用逗號分開
    f = open("/tmp/a.txt")
    lines = f.readlines()
    f.close()
    lines_new= []
    for line in lines:
        lines_new.append(line.strip())
    lines_txt=",".join(lines_new)
    print(lines_txt)
if 0: # 取代某些字
    f = open("/tmp/a.txt")
    lines = f.readlines()
    f.close()
    lines_new= []
    for line in lines:
        line=line.replace(",","")
        line=line.replace("\"","").strip()
        if line:
            lines_new.append(line)

if 0: #合併目錄裡面的所有 ods, 中的某一欄資料
    pd.set_option('display.max_rows',None)
    filenames = find('*.ods', 'output/sew')
    print(filenames)
    df_all= None
    for filename in filenames:
        sews = pd.read_excel(filename, engine="odf",sheet_name=None,skiprows=3)
        print("%s sheets:\n %s " % (filename,sews.keys()))

        df=sews['用戶接管普及率及污水處理率統計一覽表']
        df=df.rename(columns={"Unnamed: 0": "縣市"})
        #fmt="%Y%m%dT%H%M%S+8"

        basename = os.path.basename(filename)
        df['basename']=basename[:-4]
        print(df)
        if df_all is None:
            df_all = df
        else:
            df_all = pd.concat([df_all, df],axis=0)

#df_all

# 政府 API-json 範例

In [ ]:
#例子： 142979-IoW淹水深度基本資料 https://data.wra.gov.tw/Service/OpenData.aspx?format=json&id=C2C7DC74-B876-432C-8634-DF9D8DF36415

url = "https://data.wra.gov.tw/Service/OpenData.aspx?format=json&id=C2C7DC74-B876-432C-8634-DF9D8DF36415"
filename="/Users/wuulong/Downloads/test.json"
url_get(filename, url,False) #renew=False
with open(filename , 'r', encoding='utf-8-sig') as json_file: #這裡發生編碼錯誤，頭有 \ufeff，用 utf-8-sig 讀就正常
    data = json.load(json_file)
records = data['TaiwanWaterExchangingData']['HydrologyRelatedAgencyClass']['IoWFloodingDepth'] #先觀察，就會資料想要的資料在哪，再來寫 code
i=0
for r1 in records:
    print("record %i:\n%s" %(i,r1))
    i+=1
    

# 環保署 API
https://data.epa.gov.tw/api/v1

以下 API 測試過

/stat_p_123 重要河川水質概況

/wqx_p_01 河川水質監測資料

/dws_p_28 每月自來水水質監測資料

/wqx_p_12 河川水質季監測資料

/ems_s_01 環境保護許可管理系統(暨解除列管)對象基本資料

/stat_p_44 垃圾處理場(廠)統計

/stat_p_45 垃圾清理量資料

/stat_p_87 垃圾處理場(廠)座數

/ems_s_03 水污染源許可及申報資料 https://data.epa.gov.tw/dataset/ems_s_03

In [29]:
# 這裡可以將 API 資料下載下來，轉換成 df
api_key="yourkey" #yourkey 請更新成自己環保署API key
limit=1000
limit_break=0 #>0 can limit total
offset=0
load_cnt=0
renew_url=True
    
if 0: #/stat_p_123 重要河川水質概況 -> e_river_state_q
    url_t = "https://data.epa.gov.tw/api/v1/stat_p_123?offset=%i&limit=%i&api_key=%s"
    filename_t= "output/stat_p_123-%i.json"
    use_label=True
    pass
if 0: # /wqx_p_01 河川水質監測資料 -> e_river_q
    url_t = "https://data.epa.gov.tw/api/v1/wqx_p_01?offset=%i&limit=%i&api_key=%s"
    filename_t= "output/wqx_p_01-%i.json"
    use_label=False
if 0: #/dws_p_28 每月自來水水質監測資料 -> e_waterwork_q
    url_t = "https://data.epa.gov.tw/api/v1/dws_p_28?offset=%i&limit=%i&api_key=%s"
    filename_t= "output/dws_p_28-%i.json"
    use_label=False
if 0: #/wqx_p_12 河川水質季監測資料 #"total": 117041 -> e_river_season_q
    url_t = "https://data.epa.gov.tw/api/v1/wqx_p_12?offset=%i&limit=%i&api_key=%s"
    filename_t= "output/wqx_p_12-%i.json"
    use_label=False
if 0: #/ems_s_01 環境保護許可管理系統(暨解除列管)對象基本資料 = 118447-環境保護許可管理系統(暨解除列管)對象基本資料     
    #"total": 322225 -> e_factory_base
    url_t = "https://data.epa.gov.tw/api/v1/ems_s_01?offset=%i&limit=%i&api_key=%s"
    filename_t= "output/ems_s_01-%i.json"
    use_label=False
if 0: #/stat_p_44 垃圾處理場(廠)統計 "total": 220  Garbage disposal site, 不進 DB
    url_t="https://data.epa.gov.tw/api/v1/stat_p_44?api_key=%s"
    url= url_t % (api_key)
    renew_url=False
    filename_t= "output/stat_p_44-%i.json"
    use_label=False
if 0: #/stat_p_45 垃圾清理量資料 "total": 902 -> e_trash_stat_qty
    url_t = "https://data.epa.gov.tw/api/v1/stat_p_45?offset=%i&limit=%i&api_key=%s"
    filename_t= "output/stat_p_45-%i.json"
    use_label=False     
if 1: # /stat_p_46 垃圾清理回收率指標資料 "total": 2194 -> e_trash_recycle
    url_t = "https://data.epa.gov.tw/api/v1/stat_p_46?offset=%i&limit=%i&api_key=%s"
    filename_t= "output/stat_p_46-%i.json"
    use_label=False 
if 0: #/stat_p_87 垃圾處理場(廠)座數 "total": 575 -> e_garbage_disposal
    url_t = "https://data.epa.gov.tw/api/v1/stat_p_87?offset=%i&limit=%i&api_key=%s"
    filename_t= "output/stat_p_87-%i.json"
    use_label=False 
if 0: #/ems_s_03 水污染源許可及申報資料
    url_t= "https://data.epa.gov.tw/api/v1/ems_s_03?offset=%i&limit=%i&api_key=%s"
    filename_t= "output/ems_s_03-%i.json"
    use_label=False 
    
    
while True:
    if renew_url==True:
        url= url_t % (offset,limit,api_key)
    filename  = filename_t %(offset)
    print("url=%s" %(url))
    url_get(filename, url,False) #下載一個批次
    data = load_json(filename)

    if offset==0: #處理欄位名稱與知道總數量
        labels={}
        d={}
        #print("labels:")
        for i in range(len(data['fields'])):
            #print("%s=%s" %(data['fields'][i]['id'],data['fields'][i]['info']['label']))
            labels[data['fields'][i]['id']]=data['fields'][i]['info']['label']
        print("labels=%s" %(labels))
        print("total=%i" %(data['total']))

    #print("values:")
    records_cnt = len(data['records'])
    for i in range(records_cnt):
        record_cur = data['records'][i]
        #print("%i: %s " %(i, record_cur))
        for key in record_cur.keys():
            if use_label:
                if not labels[key] in d.keys():
                    #print("key=%s" %(key))
                    d[labels[key]]=[]
                d[labels[key]].append(record_cur[key])
            else:
                if not key in d.keys():
                    #print("key=%s" %(key))
                    d[key]=[]
                d[key].append(record_cur[key])
        load_cnt += 1
    #load_cnt += records_cnt
    print("load_cnt=%i" %(load_cnt))
    if load_cnt >= data['total'] or (limit_break>0 and load_cnt >= limit_break): #讀到最後一筆了
        break 
    offset += limit

df = pd.DataFrame.from_dict(d) #將資料字典轉成 dataframe
df

url=https://data.epa.gov.tw/api/v1/stat_p_46?offset=0&limit=1000&api_key=615b2545-254c-4300-8e66-c64af62d4cf0
labels={'Year': '年度', 'Month': '月份', 'County': '縣市別', 'Garbage_Recycled': '資源回收率', 'Food_Waste': '廚餘回收率', 'Bulk_Waste': '巨大垃圾回收再利用率', 'Municipal_Waste_Recycling_Rate': '垃圾回收率'}
total=2194
load_cnt=1000
url=https://data.epa.gov.tw/api/v1/stat_p_46?offset=1000&limit=1000&api_key=615b2545-254c-4300-8e66-c64af62d4cf0
load_cnt=2000
url=https://data.epa.gov.tw/api/v1/stat_p_46?offset=2000&limit=1000&api_key=615b2545-254c-4300-8e66-c64af62d4cf0
load_cnt=2194


,Year,Month,County,Garbage_Recycled,Food_Waste,Bulk_Waste,Municipal_Waste_Recycling_Rate
0,110,3,臺南市,60.64,6.60,1.13,68.37
1,110,3,連江縣,30.20,33.25,6.43,69.87
2,110,3,嘉義市,54.02,4.49,0.75,59.26
3,110,3,宜蘭縣,55.92,5.28,1.02,62.22
4,110,3,南投縣,48.49,3.99,0.98,53.46
...,...,...,...,...,...,...,...
2189,105,3,宜蘭縣,41.22%,16.83%,1.35%,59.4%
2190,105,3,高雄市,44.6%,10.4%,0.46%,55.45%
2191,105,3,臺南市,47.62%,11.23%,1.31%,60.16%
2192,105,3,臺中市,46.69%,5.28%,0.54%,52.51%


# ODS 處理範例

In [39]:
filename="include/sys.ods"
sys = pd.read_excel(filename, engine="odf",sheet_name=None,skiprows=3)
print("sheet names:")
print(sys.keys())
sys['table_def']

sheet names:
dict_keys(['description', 'table_def', 's_syspar', 's_info_line', 's_info_area', 's_timeseq_rpt', 's_repeat_job', 's_topology_kind', 's_gis_layer', 's_info_point', 's_topology_transfer', 's_topology_node', 's_topology_edge', 's_village_waterin', 's_waterin_qty', 's_waterwork_qty', 's_waterin_quality', 's_value_domain', 's_waterin_b', 's_df_brief', '污水下水道用戶接管普及率', '工作表24'])


,s_syspar,T,par_name,Unnamed: 3,Unnamed: 4
0,s_table_desc,T,table_name,NaN,NaN
1,s_info_point,T,id,name,NaN
2,s_info_line,T,id,name,NaN
3,s_info_area,T,id,name,NaN
4,s_timeseq_rpt,T,id,name,NaN
5,s_repeat_job,T,NaN,NaN,NaN
6,s_topology_kind,T,kind,NaN,NaN
7,s_topology_transfer,T,NaN,NaN,NaN
8,s_topology_node,T,node_id,node_name,NaN
9,s_topology_edge,T,edge_id,edge_name,NaN


# XML 處理範例
https://data.gov.tw/dataset/8203 全國宗教資訊系統資料-寺廟

In [13]:
#8203                                 全國宗教資訊系統資料-寺廟, KML
import pandas as pd 
import xml.etree.ElementTree as etree

tree = etree.parse("data/自開水/8203_temple.xml")
root = tree.getroot()

columns = ["寺廟名稱", "主祀神祇", "行政區","地址","教別","建別","組織型態","電話","負責人","其他","WGS84X","WGS84Y"]


df = pd.DataFrame(columns = columns)


for member in root.findall('OpenData_3'):
    cols = []
    for i in range(len(columns)):
        result = member.find(columns[i])
        cols.append(result.text if result is not None else None)
    
    df = df.append(pd.Series(cols, index = columns), ignore_index = True)

dtemple_df = df
dtemple_df

,寺廟名稱,主祀神祇,行政區,地址,教別,建別,組織型態,電話,負責人,其他,WGS84X,WGS84Y
0,竹圍仔福德祠,福德正神,臺南市,臺南市白河區大竹里14鄰大排竹206號,道教,適用監督寺廟條例寺廟,管理人(住持)制,06-6851562,錢玉珠,None,120.396797180176,23.3648853302002
1,福德祠,福德正神,臺南市,臺南市白河區糞箕湖河東里33之2號,道教,適用監督寺廟條例寺廟,管理人(住持)制,06-6852378,吳朝正,None,120.438499450684,23.3971004486084
2,永安宮,石府千歲,臺南市,臺南市麻豆區北勢里加輦邦3鄰29號,道教,適用監督寺廟條例寺廟,其他,06-5719598,郭芳財,None,120.238861083984,23.2116050720215
3,蘭陽南海宮,九天娘娘,宜蘭縣,宜蘭縣冬山鄉中山村中山二路223號,道教,適用監督寺廟條例寺廟,管理委員會制,03-9587103,何登和,None,121.744438171387,24.6360340118408
4,花蓮般若精舍,釋迦牟尼佛,花蓮縣,花蓮縣瑞穗鄉富興村154-1號,佛教,適用監督寺廟條例寺廟,管理人(住持)制,03-8876657,釋印可,None,121.387298583984,23.5796985626221
...,...,...,...,...,...,...,...,...,...,...,...,...
11899,仙興寺,觀音佛祖,嘉義縣,嘉義縣中埔鄉嘉義縣中埔鄉富收村18鄰興化廍18號附2,道教,適用監督寺廟條例寺廟,管理委員會制,05-2533321,郭照期,None,120.475738525391,23.4430503845215
11900,財團法人平山宮,五顯大帝.媽祖,南投縣,南投縣草屯鎮坪頂里南平路27號,道教,適用監督寺廟條例寺廟,管理委員會制,049-2552468,None,None,120.74690246582,23.9673099517822
11901,財團法人草屯巧聖先師廟,巧聖仙師,南投縣,南投縣草屯鎮御史里玉成路49巷20號,道教,適用監督寺廟條例寺廟,None,049-2313488,None,None,120.698600769043,23.9924392700195
11902,test,阿彌佗佛,宜蘭縣,宜蘭縣羅東鎮中興路3號,佛教,適用監督寺廟條例寺廟,None,-00000000,None,None,121.766975402832,24.676944732666


# 存取 SHP

In [38]:
filename="/Users/wuulong/github/waterswak/data/7442-直轄市、縣市界線(TWD97經緯度)/COUNTY_MOI.shp"
df = gpd.read_file(filename)
df

,COUNTYID,COUNTYCODE,COUNTYNAME,COUNTYENG,geometry
0,Z,09007,連江縣,Lienchiang County,"MULTIPOLYGON (((119.96445 25.94552, 119.96427 ..."
1,G,10002,宜蘭縣,Yilan County,"MULTIPOLYGON (((121.95971 24.84494, 121.96064 ..."
2,N,10007,彰化縣,Changhua County,"POLYGON ((120.45655 24.20719, 120.48589 24.197..."
3,M,10008,南投縣,Nantou County,"POLYGON ((121.27086 24.23661, 121.27108 24.236..."
4,P,10009,雲林縣,Yunlin County,"MULTIPOLYGON (((120.08108 23.52412, 120.08161 ..."
5,C,10017,基隆市,Keelung City,"MULTIPOLYGON (((121.71015 25.17604, 121.71015 ..."
6,A,63000,臺北市,Taipei City,"POLYGON ((121.57098 25.19717, 121.57097 25.197..."
7,F,65000,新北市,New Taipei City,"POLYGON ((121.53752 25.30003, 121.53768 25.300..."
8,B,66000,臺中市,Taichung City,"POLYGON ((121.32834 24.43304, 121.32845 24.432..."
9,D,67000,臺南市,Tainan City,"POLYGON ((120.44025 23.41308, 120.44040 23.413..."
